In [1]:
import os
import pandas as pd

base_path = r"C:\Users\HP\Desktop\data_science\primo_anno\DataViz\Csv Progetto"

files = {
    "standard":   "Squad Standard Stats.csv",
    "passing":    "Squad Passing.csv",
    "shooting":   "Squad Shooting.csv",
    "gca":        "Squad Goal and Shot Creation.csv",
    "possession": "Squad Possession.csv",
    "defense":    "Squad Defensive Actions.csv",
}

def load_fbref_csv(filename):
    path = os.path.join(base_path, filename)
    df = pd.read_csv(
        path,
        sep=";",        # separator used in the file
        header=1,       # real header is on row 2
        engine="python"
    )
    df = df.applymap(lambda x: str(x).replace(",", ".") if isinstance(x, str) else x)
    for col in df.columns:
        if col != "Squad":  # keep team name as string
            df[col] = pd.to_numeric(df[col], errors="ignore")
    return df

# carica tutte le tabelle 
tables = {name: load_fbref_csv(fname) for name, fname in files.items()}
df_standard   = tables["standard"]
df_passing    = tables["passing"]
df_shooting   = tables["shooting"]
df_gca        = tables["gca"]
df_possession = tables["possession"]
df_defense    = tables["defense"]

# carica OVERALL separato, con header corretto
overall_path = os.path.join(base_path, "Serie A overall.csv")
df_overall = pd.read_csv(
    overall_path,
    sep=";",
    header=0,      # seconda riga = header, nessuna riga di dati persa
    engine="python"
)
df_overall = df_overall.applymap(lambda x: str(x).replace(",", ".") if isinstance(x, str) else x)
for col in df_overall.columns:
    if col != "Squad":
        df_overall[col] = pd.to_numeric(df_overall[col], errors="ignore")

# se vuoi, piccolo rename compatto (non obbligatorio se i nomi già ti vanno bene)
df_overall = df_overall.rename(columns={
    "Rk":"Rk","Squad":"Squad","MP":"MP","W":"W","D":"D","L":"L",
    "GF":"GF","GA":"GA","GD":"GD","Pts":"Pts","Pts/MP":"Pts/MP",
    "xG":"xG","xGA":"xGA","xGD":"xGD","xGD/90":"xGD/90",
    "Attendance":"Attendance","Top Team Scorer":"Top Team Scorer",
    "Goalkeeper":"Goalkeeper","Notes":"Notes"
})

# rename delle altre tabelle (come avevi già)
df_standard.rename(columns={
    'Gls.1': 'Gls_x90', 'Ast.1': 'Ast_x90', 'G+A.1': 'G+Ax90',
    'G-PK.1': 'G-PKx90', 'xG.1':'xG_90', 'xAG.1': 'xAG_90',
    'npxG.1': 'npxG_90', 'npxG+xAG.1': 'npxG+xAG_90'
}, inplace=True)

df_possession.rename(columns={'Def 3rd': 'Def_3rd', 'Mid 3rd': 'Mid_3rd', 'Att 3rd': 'Att_3rd','01-mar': 'Carries_into_last_1/3'}, inplace=True)

df_defense.rename(columns={'Def 3rd': 'Def_3rd', 'Mid 3rd': 'Mid_3rd', 'Att 3rd': 'Att_3rd'}, inplace=True)

df_passing.rename(columns={
    'Cmp.1': 'short_Cmp', 'Att.1': 'short_Att', 'Cmp%.1': 'short_Cmp%',
    'Cmp.2': 'med_Cmp',   'Att.2': 'med_Att',  'Cmp%.2': 'med_Cmp%',
    'Cmp.3': 'long_Cmp',  'Att.3': 'long_Att', 'Cmp%.3': 'long_Cmp%',
    '01-mar': 'passes_into_last_1/3'
}, inplace=True)

df_gca.rename(columns={
    'PassLive':   'Pass_Leading_to_Shot_nodeadballs',
    'PassDead':   'Passes_from_deadballs_Leading_to_Shot',
    'TO':         'dribbl_lead_to_shot',
    'Fld':        'Foul_Drawn_toShot',
    'Def':        'Def_action_toShot',
    'PassLive.1': 'Pass_Leading_to_Goal_nodeadballs',
    'PassDead.1': 'Passes_from_deadballs_Leading_to_Goal',
    'TO.1':       'dribbl_lead_to_goal',
    'Fld.1':      'Foul_Drawn_toGoal',
    'Def.1':      'Def_action_toGoal'
}, inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_21220\1447516937.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).replace(",", ".") if isinstance(x, str) else x)
C:\Users\HP\AppData\Local\Temp\ipykernel_21220\1447516937.py:26: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\HP\AppData\Local\Temp\ipykernel_21220\1447516937.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).replace(",", ".") if isinstance(x, str) else x)
C:\Users\HP\AppData\Local\Temp\ipykernel_21220\1447516937.py:26: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col],

## Preprocessing and variable selection

In [23]:
df_standard.head()

,Squad,# Pl,Age,Poss,MP,Starts,Min,90s,Gls,Ast,...,Gls_x90,Ast_x90,G+Ax90,G-PKx90,G+A-PK,xG_90,xAG_90,xG+xAG,npxG_90,npxG+xAG_90
0,Atalanta,32,27.1,55.3,38,418,3.42,38.0,76,54,...,2.00,1.42,3.42,1.87,3.29,1.71,1.23,2.94,1.57,2.80
1,Bologna,30,26.4,58.1,38,418,3.42,38.0,56,43,...,1.47,1.13,2.61,1.32,2.45,1.30,0.93,2.22,1.15,2.08
2,Cagliari,28,26.6,44.9,38,418,3.42,38.0,38,31,...,1.00,0.82,1.82,0.92,1.74,1.12,0.84,1.97,1.06,1.90
3,Como,38,26.3,54.6,38,418,3.42,38.0,44,39,...,1.16,1.03,2.18,1.16,2.18,1.20,0.96,2.16,1.16,2.12
4,Empoli,34,25.4,41.7,38,418,3.42,38.0,31,21,...,0.82,0.55,1.37,0.76,1.32,0.86,0.60,1.46,0.79,1.40


In [17]:
df_defense.head()

,Squad,# Pl,90s,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Att,Tkl%,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err
0,Atalanta,32,38.0,571,347,238,260,73,272,506,53.8,234,360,98,262,296,867,679,16
1,Bologna,30,38.0,591,346,215,270,106,283,538,52.6,255,390,93,297,258,849,679,20
2,Cagliari,28,38.0,538,303,264,224,50,238,449,53.0,211,343,131,212,276,814,887,18
3,Como,38,38.0,663,412,301,260,102,303,591,51.3,288,348,91,257,279,942,788,19
4,Empoli,34,38.0,689,396,314,293,82,309,543,56.9,234,396,136,260,323,1012,1072,21


In [18]:
df_gca.head()

,Squad,# Pl,90s,SCA,SCA90,Pass_Leading_to_Shot_nodeadballs,Passes_from_deadballs_Leading_to_Shot,dribbl_lead_to_shot,Sh,Foul_Drawn_toShot,Def_action_toShot,GCA,GCA90,Pass_Leading_to_Goal_nodeadballs,Passes_from_deadballs_Leading_to_Goal,dribbl_lead_to_goal,Sh.1,Foul_Drawn_toGoal,Def_action_toGoal
0,Atalanta,32,38.0,966,25.42,736,75,51,71,22,11,131,3.45,105,5,4,13,4,0
1,Bologna,30,38.0,913,24.03,666,84,39,53,51,20,103,2.71,78,3,3,9,7,3
2,Cagliari,28,38.0,748,19.68,516,88,36,66,34,8,71,1.87,54,4,2,5,5,1
3,Como,38,38.0,920,24.21,673,84,44,66,40,13,79,2.08,66,2,5,3,1,2
4,Empoli,34,38.0,638,16.79,452,53,41,35,39,18,51,1.34,35,4,5,4,2,1


In [17]:
df_overall.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Napoli,38,24,10,4,59,27,32,82,2.16,52.9,29.6,23.3,0.61,50.909,Romelu Lukaku - 14,Alex Meret,→ Champions League via league finish
1,2,Inter,38,24,9,5,79,35,44,81,2.13,66.9,36.4,30.5,0.80,70.126,Marcus Thuram - 14,Yann Sommer,→ Champions League via league finish
2,3,Atalanta,38,22,8,8,78,37,41,74,1.95,65.0,38.6,26.4,0.69,22.675,Mateo Retegui - 25,Marco Carnesecchi,→ Champions League via league finish
3,4,Juventus,38,18,16,4,58,35,23,70,1.84,51.0,34.0,16.9,0.45,40.275,Dušan Vlahović - 10,Michele Di Gregorio,→ Champions League via league finish
4,5,Roma,38,20,9,9,56,35,21,69,1.82,53.0,40.9,12.1,0.32,61.895,Artem Dovbyk - 12,Mile Svilar,→ Europa League via league finish


In [16]:
df_passing.head()

,Squad,# Pl,90s,Cmp,Att,Cmp%,TotDist,PrgDist,short_Cmp,short_Att,...,long_Cmp%,Ast,xAG,xA,A-xAG,KP,passes_into_last_1/3,PPA,CrsPA,PrgP
0,Atalanta,32,38.0,17454,20998,83.1,304878,97688,7796,8669,...,59.6,54,46.9,46.4,7.1,425,1382,395,113,1824
1,Bologna,30,38.0,15681,19336,81.1,295869,97686,6113,6881,...,57.5,43,35.2,30.7,7.8,384,1351,293,109,1468
2,Cagliari,28,38.0,12092,15802,76.5,219114,88994,5258,6076,...,48.4,31,32.0,29.3,-1.0,301,907,234,89,1150
3,Como,38,38.0,15917,19183,83.0,268133,95134,7607,8351,...,54.5,39,36.6,30.8,2.4,405,1150,279,60,1377
4,Empoli,34,38.0,10093,13916,72.5,186741,74734,4015,4844,...,45.0,21,22.9,19.4,-1.9,269,799,214,71,1013


In [15]:
df_possession.head()

,Squad,# Pl,Poss,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,...,Carries,TotDist,PrgDist,PrgC,Carries_into_last_1/3,CPA,Mis,Dis,Rec,PrgR
0,Atalanta,32,55.3,38.0,25020,2035,6907,11325,7007,1076,...,15076,80503,42064,897,696,254,574,362,17329,1808
1,Bologna,30,58.1,38.0,23304,2070,6703,10984,5817,831,...,13684,70992,33468,639,422,167,598,361,15512,1457
2,Cagliari,28,44.9,38.0,19802,2531,7637,8203,4166,645,...,10679,52998,25903,506,339,115,585,294,11974,1140
3,Como,38,54.6,38.0,23246,2359,7668,10461,5315,791,...,14108,70678,35128,658,463,202,577,360,15729,1371
4,Empoli,34,41.7,38.0,18255,2279,6509,8088,3861,562,...,8486,43445,20518,428,284,102,594,349,9986,1004


In [14]:
df_shooting.head()

,Squad,# Pl,90s,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
0,Atalanta,32,38.0,76,538,181,33.6,14.16,4.76,0.13,0.39,15.0,5,5,7,65.0,59.5,0.11,11.0,11.5
1,Bologna,30,38.0,56,505,166,32.9,13.29,4.37,0.10,0.30,17.5,18,6,7,49.3,43.8,0.09,6.7,6.2
2,Cagliari,28,38.0,38,421,115,27.3,11.08,3.03,0.08,0.30,16.0,10,3,3,42.7,40.3,0.10,-4.7,-5.3
3,Como,38,38.0,44,519,176,33.9,13.66,4.63,0.08,0.25,17.9,17,0,2,45.4,43.9,0.09,-1.4,0.1
4,Empoli,34,38.0,31,367,107,29.2,9.66,2.82,0.08,0.27,19.1,21,2,3,32.5,30.1,0.08,-1.5,-1.1


In [3]:
df_overall.columns

Index(['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP',
       'xG', 'xGA', 'xGD', 'xGD/90', 'Attendance', 'Top Team Scorer',
       'Goalkeeper', 'Notes'],
      dtype='object')

In [18]:
# -----------------------------
# 1. COLONNE DA TENERE
# -----------------------------

cols_standard = ["Squad", "xG_90", "npxG_90", 'Gls_x90']
cols_passing = ["Squad", "Cmp", "Cmp%","short_Cmp", "med_Cmp", "long_Cmp", "PrgDist", "PrgP", "KP", "passes_into_last_1/3", "PPA", "CrsPA"]
#add percentage short% med% long% = short_cmp/cmp etc
cols_shooting = ["Squad", "SoT/90", "G/SoT", "SoT%", "Dist","npxG", "npxG/Sh", "G-xG"]
cols_gca = ["Squad", "SCA90", "GCA90", "Pass_Leading_to_Shot_nodeadballs", "Passes_from_deadballs_Leading_to_Shot", "dribbl_lead_to_shot", "Foul_Drawn_toShot", "Def_action_toShot",
    "Pass_Leading_to_Goal_nodeadballs", "Passes_from_deadballs_Leading_to_Goal", "dribbl_lead_to_goal", "Foul_Drawn_toGoal","Def_action_toGoal"] 
cols_possession = ["Squad",  "Poss", "Touches",  "Def_3rd", "Mid_3rd", "Att_3rd", "Att Pen", "Succ%", "Carries", "TotDist", "PrgDist", "PrgC", "Carries_into_last_1/3", "PrgR"]
#add touches_x90, PrgC_x90, PrgR_x90
cols_defense = ["Squad", "Tkl", "TklW", "Int", "Tkl+Int", "Def_3rd", "Mid_3rd", "Att_3rd"] #add tklW%
cols_overall = ["Rk", "Squad", "Pts", "Pts/MP", "GA", "xGA", "xGD", "xGD/90", "GD"]
#Occhio a Variabili con lo stesso nome in df diversi, es: "Cmp" in passing e "Cmp" in defense 
# -----------------------------
# 2. FILTRARE I DATAFRAME
# -----------------------------

df_standard_f = df_standard[cols_standard].copy()
df_passing_f = df_passing[cols_passing].copy()
df_shooting_f = df_shooting[cols_shooting].copy()
df_gca_f = df_gca[cols_gca].copy()
df_possession_f = df_possession[cols_possession].copy()
df_defense_f = df_defense[cols_defense].copy()
df_overall_f = df_overall[cols_overall].copy()
df_overall_f["Squad"] = (
    df_overall_f["Squad"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)  # rimuove NBSP
    .str.strip()                            # rimuove spazi iniziali/finali
)

In [19]:
#add percentage short% med% long% = short_cmp/cmp etc
df_passing_f['short_Cmp%'] = df_passing_f['short_Cmp'] / df_passing_f['Cmp'] * 100
df_passing_f['med_Cmp%'] = df_passing_f['med_Cmp'] / df_passing_f['Cmp'] * 100
df_passing_f['long_Cmp%'] = df_passing_f['long_Cmp'] / df_passing_f['Cmp'] * 100
df_defense_f['TKlW%'] = df_defense_f['TklW'] / df_defense_f['Tkl']

In [20]:
#add touches_x90, PrgC_x90, PrgR_x90
df_possession_f['touches_x90'] = df_possession_f['Touches'] / 38
df_possession_f['PrgC_x90'] = df_possession_f['PrgC'] / 38
df_possession_f['PrgR_x90'] = df_possession_f['PrgR'] / 38
df_possession_f.head()

,Squad,Poss,Touches,Def_3rd,Mid_3rd,Att_3rd,Att Pen,Succ%,Carries,TotDist,PrgDist,PrgC,Carries_into_last_1/3,PrgR,touches_x90,PrgC_x90,PrgR_x90
0,Atalanta,55.3,25020,6907,11325,7007,1076,39.8,15076,80503,42064,897,696,1808,658.421053,23.605263,47.578947
1,Bologna,58.1,23304,6703,10984,5817,831,39.8,13684,70992,33468,639,422,1457,613.263158,16.815789,38.342105
2,Cagliari,44.9,19802,7637,8203,4166,645,41.3,10679,52998,25903,506,339,1140,521.105263,13.315789,30.000000
3,Como,54.6,23246,7668,10461,5315,791,42.4,14108,70678,35128,658,463,1371,611.736842,17.315789,36.078947
4,Empoli,41.7,18255,6509,8088,3861,562,42.0,8486,43445,20518,428,284,1004,480.394737,11.263158,26.421053


In [21]:
df_overall_f['xGA_90'] = df_overall_f['xGA'] / 38
df_overall_f['GA_x90'] = df_overall_f['GA'] / 38
df_overall_f.head()

,Rk,Squad,Pts,Pts/MP,GA,xGA,xGD,xGD/90,GD,xGA_90,GA_x90
0,1,Napoli,82,2.16,27,29.6,23.3,0.61,32,0.778947,0.710526
1,2,Inter,81,2.13,35,36.4,30.5,0.80,44,0.957895,0.921053
2,3,Atalanta,74,1.95,37,38.6,26.4,0.69,41,1.015789,0.973684
3,4,Juventus,70,1.84,35,34.0,16.9,0.45,23,0.894737,0.921053
4,5,Roma,69,1.82,35,40.9,12.1,0.32,21,1.076316,0.921053


In [22]:
df_possession_f.rename(columns={'PrgDist': 'PrgDist_carry', 'Def_3rd': 'Def_3rd_touches', 'Mid3rd': 'Mid_3rd_touches', 'Att_3rd': 'Att_3rd_touches'}, inplace=True)
df_defense_f.rename(columns = {'Def_3rd': 'Def_3rd_tackles', 'Mid_3rd': 'Mid_3rd_tackles', 'Att_3rd': 'Att_3rd_tackles'}, inplace=True)
df_passing_f.rename(columns={'short_Cmp': 'short_Cmp', 'med_Cmp': 'med_Cmp', 'long_Cmp': 'long_Cmp', 'PrgDist': 'PrgDist_pass'}, inplace=True)

In [16]:
df_defense_f.head()

,Squad,Tkl,TklW,Int,Tkl+Int,Def_3rd_tackles,Mid_3rd_tackles,Att_3rd_tackles,TKlW%
0,Atalanta,571,347,296,867,238,260,73,0.607706
1,Bologna,591,346,258,849,215,270,106,0.585448
2,Cagliari,538,303,276,814,264,224,50,0.563197
3,Como,663,412,279,942,301,260,102,0.621418
4,Empoli,689,396,323,1012,314,293,82,0.574746


## Merge

In [23]:
df_teams = df_overall_f.copy()  
dfs_to_merge = [
    df_standard_f,
    df_passing_f,
    df_shooting_f,
    df_gca_f,
    df_possession_f,
    df_defense_f]
for df in dfs_to_merge:
    df_teams = df_teams.merge(df, on="Squad", how="left")
print(df_teams.shape)
print(df_teams.head())

(20, 71)
   Rk     Squad  Pts  Pts/MP  GA   xGA   xGD  xGD/90  GD    xGA_90  ...  \
0   1    Napoli   82    2.16  27  29.6  23.3    0.61  32  0.778947  ...   
1   2     Inter   81    2.13  35  36.4  30.5    0.80  44  0.957895  ...   
2   3  Atalanta   74    1.95  37  38.6  26.4    0.69  41  1.015789  ...   
3   4  Juventus   70    1.84  35  34.0  16.9    0.45  23  0.894737  ...   
4   5      Roma   69    1.82  35  40.9  12.1    0.32  21  1.076316  ...   

    PrgC_x90   PrgR_x90  Tkl  TklW  Int  Tkl+Int  Def_3rd_tackles  \
0  19.447368  40.263158  535   323  270      805              237   
1  18.263158  45.184211  505   288  227      732              215   
2  23.605263  47.578947  571   347  296      867              238   
3  21.315789  40.736842  606   353  215      821              273   
4  18.184211  39.157895  551   328  282      833              233   

   Mid_3rd_tackles  Att_3rd_tackles     TKlW%  
0              238               60  0.603738  
1              198           

In [18]:
df_teams.columns

Index(['Rk', 'Squad', 'Pts', 'Pts/MP', 'xGD', 'xGD/90', 'GD', 'xG_90',
       'npxG_90', 'Gls_x90', 'Cmp', 'Cmp%', 'short_Cmp', 'med_Cmp', 'long_Cmp',
       'PrgDist_pass', 'PrgP', 'KP', 'passes_into_last_1/3', 'PPA', 'CrsPA',
       'short_Cmp%', 'med_Cmp%', 'long_Cmp%', 'SoT/90', 'G/SoT', 'SoT%',
       'Dist', 'npxG', 'npxG/Sh', 'G-xG', 'SCA90', 'GCA90',
       'Pass_Leading_to_Shot_nodeadballs',
       'Passes_from_deadballs_Leading_to_Shot', 'dribbl_lead_to_shot',
       'Foul_Drawn_toShot', 'Def_action_toShot',
       'Pass_Leading_to_Goal_nodeadballs',
       'Passes_from_deadballs_Leading_to_Goal', 'dribbl_lead_to_goal',
       'Foul_Drawn_toGoal', 'Def_action_toGoal', 'Poss', 'Touches',
       'Def_3rd_touches', 'Mid_3rd', 'Att_3rd_touches', 'Att Pen', 'Succ%',
       'Carries', 'TotDist', 'PrgDist_carry', 'PrgC', 'Carries_into_last_1/3',
       'PrgR', 'touches_x90', 'PrgC_x90', 'PrgR_x90', 'Tkl', 'TklW', 'Int',
       'Tkl+Int', 'Def_3rd_tackles', 'Mid_3rd_tackles', 'Att_3r

## Download

In [24]:
df_teams.to_csv(os.path.join(base_path, "SerieA_2425stats_merged.csv"), index=False)